In this Notebook I had a look at the sequence with a particulary high loss on the single view_direction image model. By this I could detect a some false labels.

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds
import tqdm
import matplotlib.pyplot as plt

# Here I just loaded the training images and inserted some addiational info to the df

# df = pd.read_feather("traning_imgs.df")
# old = pd.read_feather("../data_analysis/prelim.feather").set_index("key")
# df["seq_key"] = old.loc[df["key"].values,"sequence_key"].values
# df["view_direction"] = old.loc[df["key"].values,"view_direction"].values
# df["city"] = df.path.apply(lambda x: x[22: x.find("/",22)])
# func = lambda s: s[s.find("images/")+7:]
# df["local_path"] =  "../seq_data/" + df.ds_type + "/" + df.path.apply(func).values
# df = df.sort_values(["city","seq_key","og_cluster"])
# df.index = range(len(df))
# df.to_feather("traning_imgs.df")

2022-08-23 11:03:28.766867: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-23 11:03:28.766912: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/felix/anaconda3/envs/r/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


This was for calculating the losses and save them

In [2]:
# # ----------- Get Predictions -----------

# df = pd.read_feather("traning_imgs.df")

# model = tf.keras.models.load_model("../../view_directions_task_single_image/Training/training_results/last_model/ENB0_All/model")
# def mapping(path,label):

#     image = tf.io.read_file(path)
#     image = tf.io.decode_jpeg(image, channels=3)
#     label = tf.one_hot(label,2)
#     return image, label
# ds = tf.data.Dataset.from_tensor_slices((df.local_path.values,(df.view_direction=="Sideways").apply(int).values)).map(mapping).batch(64)

# preds = model.predict(ds,verbose=1) # predict 

# # ----------- Save them to the df -----------

# df["lon"] = preds[:,0]
# df["lat"] = preds[:,1]
# df.to_feather('imgs_preds.df')

# df = pd.read_feather('imgs_preds.df').sort_values(["city",'seq_key','og_cluster'])
# labels = tf.one_hot((df.view_direction == "Sideways").apply(int).values,2).numpy()
# preds = np.array([df.lon,df.lat]).T

# df['label0'] = labels[:,0]
# df['label1'] = labels[:,1]

# # ----------- Calculate loss by cluster -----------


# index = []
# vals = []
# bce = tf.keras.losses.BinaryCrossentropy(from_logits=False)
# for i in tqdm.tqdm(df.groupby(["city",'seq_key','og_cluster'])):

#     preds = np.array([i[1].lon.values, i[1].lat.values]).T
#     labels = np.array([i[1].label0.values, i[1].label1.values]).T
    
#     loss = bce(labels,preds).numpy()
#     vals.append(loss)
#     index.append(i[0])

# # ----------- Save values -----------


# losses = pd.DataFrame(vals,index=pd.MultiIndex.from_tuples(index),columns=['loss'])
# losses.sort_values("loss",ascending=False,inplace=True)
# losses.reset_index().to_feather('losses.df')


### Load saved Losses

In [6]:
losses = pd.read_feather("losses.df").set_index(["level_0","level_1","level_2"])
df = pd.read_feather('imgs_preds.df').sort_values(["city",'seq_key','og_cluster'])
losses.index.names = ['city', 'seq_key', 'og_cluster']
paths = df.groupby(["city",'seq_key','og_cluster'])["local_path"].unique()
view = df.groupby(["city",'seq_key','og_cluster'])["view_direction"].min()
losses = losses.join(paths).join(view)
losses.head(3)

loss  \
city   seq_key                og_cluster              
moscow 88f4LVGNDL4iRFqim7cxag 1690        15.379092   
       8YtHW9q4gIQLorhQdIfZTQ 3015        15.058754   
paris  xsqtzd8zs5kp51plyijhxm 11          14.708673   

                                                                                 local_path  \
city   seq_key                og_cluster                                                      
moscow 88f4LVGNDL4iRFqim7cxag 1690        [../seq_data/test/vA89Ip9RxWNHA28ZUHgPlA.jpg, ...   
       8YtHW9q4gIQLorhQdIfZTQ 3015        [../seq_data/test/4oBMNU3mQGNI0xN-rAkOQw.jpg, ...   
paris  xsqtzd8zs5kp51plyijhxm 11          [../seq_data/train/Proo9t0uBay02aQaytJ_9A.jpg,...   

                                         view_direction  
city   seq_key                og_cluster                 
moscow 88f4LVGNDL4iRFqim7cxag 1690             Sideways  
       8YtHW9q4gIQLorhQdIfZTQ 3015             Sideways  
paris  xsqtzd8zs5kp51plyijhxm 11                Forward

Number of cluster with average loss over 2

In [7]:
((2 < losses.loss) ).sum()

158

Save Images of Losses

In [8]:
# high_loss = losses[(2 < losses.loss) ]
# for n in tqdm.tqdm(range(len(high_loss))):

#     row = high_loss.iloc[n]
#     fig, ax = plt.subplots(4,4,figsize=(12,12))
#     keys = row['local_path']
#     #np.random.shuffle(keys)
#     for i,a in enumerate(ax.flatten()):
#         try:
#             a.imshow(plt.imread(keys[i]))
#         except: pass
#         a.axis('off')
#     plt.suptitle(f"{row['view_direction']} {row['loss'] :.2f}")
#     path = "hl/" + "|".join([str(u) for u in high_loss.index[n]]) + ".jpg"
#     plt.savefig(path)
#     plt.close()


Some Utils

In [9]:
def plot_row(row):
    fig, ax = plt.subplots(4,3,figsize=(8,8))
    keys = row['local_path']
    np.random.shuffle(keys)
    for i,a in enumerate(ax.flatten()):
        try:
            a.imshow(plt.imread(keys[i]))
        except:
            pass
        a.axis('off')
    plt.suptitle(f"{row['view_direction']} {row['loss'] :.2f}")
    plt.show()

def reindex(x):
    x = x[:-4].split("|")
    x[2] = int(x[2])
    return tuple(x)


Relabel the data

In [10]:
# get list of indices

relabel_to_forward = [reindex(seq) for seq in os.listdir("lon") if losses.loc[reindex(seq),"view_direction"] == "Sideways"]
relabel_to_side = [reindex(seq) for seq in os.listdir("side") if losses.loc[reindex(seq),"view_direction"] != "Sideways"]
to_trop = [reindex(i) for i in os.listdir("del")]


def get_indices_fast(props):

    agg = []
    for i in relabel_to_forward:

        k = df.copy()
        k = k.loc[k.city==i[0]]
        k = k.loc[k.seq_key ==i[1]]
        k = k.loc[k.og_cluster == i[2]]
        agg.append(k.index.values)
    
    return np.concatenate(agg)

# -------------- Relabel images --------------
to_front = get_indices_fast(relabel_to_forward)
to_side = get_indices_fast(relabel_to_side)
df.loc[to_front,"view_direction"] = "Forward"
df.loc[to_side,"view_direction"] = "Sideways"

# -------------- Relabel Seqs  --------------

df = df.set_index("key")
seqs_df = pd.read_feather("training.df").set_index("index")
seqs_df["view_direction"] = seqs_df["keys"].apply(lambda x: df.loc[x[0],"view_direction"])

# -------------- Drop bad seqs --------------

to_del = get_indices_fast(to_trop)
trash = seqs_df["keys"].apply(lambda x: x[0] in to_del)
seqs_df = seqs_df [ ~trash]
df.drop(index = to_del,inplace=True)

Save cleaned seqs

In [12]:
df.reset_index().to_feather("cleaned_images.df")
seqs_df.reset_index().to_feather("cleaned_seqs.df")